In [8]:
import cv2
import numpy as np
import csv
import os

### Краткое описание кода:

Данный код представляет собой скрипт на языке Python с использованием библиотеки OpenCV для обработки изображений. Он предназначен для измерения высоты растений на двух изображениях (сверху и спереди) и сопоставления этих растений между двумя днями.

### Документация:

#### Загрузка изображений:

```python
image_top = cv2.imread("путь_к_изображению_сверху.jpg")
image_front = cv2.imread("путь_к_изображению_спереди.jpg")
```

Загружает изображения сверху и спереди.

#### Предварительная обработка изображений:

```python
# Преобразование изображений в цветовое пространство RGB
image_top_rgb = cv2.cvtColor(image_top, cv2.COLOR_BGR2RGB)
image_front_rgb = cv2.cvtColor(image_front, cv2.COLOR_BGR2RGB)

# Определение диапазона зеленого цвета
lower_green = (0, 100, 0)
upper_green = (100, 255, 100)

# Применение пороговой обработки для выделения зеленых пикселей
mask_top = cv2.inRange(image_top_rgb, lower_green, upper_green)
mask_front = cv2.inRange(image_front_rgb, lower_green, upper_green)

# Применение операции морфологического закрытия для удаления шума
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
closed_top = cv2.morphologyEx(mask_top, cv2.MORPH_CLOSE, kernel)
closed_front = cv2.morphologyEx(mask_front, cv2.MORPH_CLOSE, kernel)
```

Выполняет преобразование цветов, определяет диапазон зеленого цвета, применяет пороговую обработку и операцию морфологического закрытия для улучшения качества изображений.

#### Измерение высот растений:

```python
# Поиск контуров зеленых регионов
contours_top, _ = cv2.findContours(closed_top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_front, _ = cv2.findContours(closed_front, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание тензоров для сохранения высот растков
heights_top = np.zeros(len(contours_top))
heights_front = np.zeros(len(contours_front))

# Измерение высот растков и сохранение в тензоры
for i, contour_top in enumerate(contours_top):
    height_top = cv2.contourArea(contour_top)
    heights_top[i] = height_top

for i, contour_front in enumerate(contours_front):
    height_front = cv2.contourArea(contour_front)
    heights_front[i] = height_front
```

Находит контуры зеленых регионов и измеряет высоты растений, сохраняя результаты в тензоры.

#### Вывод результатов:

```python
# Вывод измеренных высот растений
print("Высоты растков на изображении сверху:", heights_top)
print("Высоты растков на изображении спереди:", heights_front)
```

Выводит измеренные высоты растений на первый день.

### Сопоставление растений между днями:

```python
# Сопоставление контуров между днями
contour_matches_top = match_contours(contours_top, contours_top_next_day)

# Вывод результатов сопоставления
print("\nРезультаты сопоставления контуров (сверху):")
for match in contour_matches_top:
    contour_curr, contour_prev, distance = match
    print(f"Контур текущего дня: {contour_curr}, Контур предыдущего дня: {contour_prev}, Расстояние: {distance}")
```

Сопоставляет контуры растений между текущим и следующим днем с выводом результатов.

In [9]:
def match_contours(contours_prev, contours_current):
    matches = []  # Список пар соответствующих контуров

    for contour_curr in contours_current:
        best_match = None
        min_distance = float('inf')

        for contour_prev in contours_prev:
            distance = cv2.matchShapes(contour_curr, contour_prev, cv2.CONTOURS_MATCH_I2, 0.0)
            if distance < min_distance:
                min_distance = distance
                best_match = contour_prev

        matches.append((contour_curr, best_match, min_distance))

    return matches

# Загрузка изображений для изначальной оценки
image_top = cv2.imread("d0UPNptCkJs.jpg")
image_front = cv2.imread("pHCk-s6FJls.jpg")

# Преобразование изображений в цветовое пространство RGB
image_top_rgb = cv2.cvtColor(image_top, cv2.COLOR_BGR2RGB)
image_front_rgb = cv2.cvtColor(image_front, cv2.COLOR_BGR2RGB)

# Определение диапазона зеленого цвета
lower_green = (0, 100, 0)
upper_green = (100, 255, 100)

# Применение пороговой обработки для выделения зеленых пикселей на изображении сверху
mask_top = cv2.inRange(image_top_rgb, lower_green, upper_green)

# Применение пороговой обработки для выделения зеленых пикселей на изображении спереди
mask_front = cv2.inRange(image_front_rgb, lower_green, upper_green)

# Применение операции морфологического закрытия для удаления шума на изображении сверху
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
closed_top = cv2.morphologyEx(mask_top, cv2.MORPH_CLOSE, kernel)

# Применение операции морфологического закрытия для удаления шума на изображении спереди
closed_front = cv2.morphologyEx(mask_front, cv2.MORPH_CLOSE, kernel)

# Поиск контуров зеленых регионов на изображении сверху
contours_top, _ = cv2.findContours(closed_top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Поиск контуров зеленых регионов на изображении спереди
contours_front, _ = cv2.findContours(closed_front, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание тензоров для сохранения высот растков
heights_top = np.zeros(len(contours_top))
heights_front = np.zeros(len(contours_front))

# Измерение высот растков и сохранение в тензоры
for i, contour_top in enumerate(contours_top):
    # Измерение высоты растка на изображении сверху
    height_top = cv2.contourArea(contour_top)
    heights_top[i] = height_top

for i, contour_front in enumerate(contours_front):
    # Измерение высоты растка на изображении спереди
    height_front = cv2.contourArea(contour_front)
    heights_front[i] = height_front

# Вывод измеренных высот растков на первый день
print("Высоты растков на первый день (сверху):", heights_top)
print("Высоты растков на первый день (спереди):", heights_front)

Высоты растков на первый день (сверху): [ 0.   0.   0.  ...  0.   0.  45.5]
Высоты растков на первый день (спереди): [0. 0. 1. ... 3. 0. 0.]


In [10]:
# Загрузка изображений для текущего дня
image_top = cv2.imread("d0UPNptCkJs.jpg")
image_front = cv2.imread("pHCk-s6FJls.jpg")

# Загрузка изображений для следующего дня
image_top_next_day = cv2.imread("d0UPNptCkJs.jpg")
image_front_next_day = cv2.imread("pHCk-s6FJls.jpg")

# Преобразование изображений в цветовое пространство RGB
image_top_rgb = cv2.cvtColor(image_top, cv2.COLOR_BGR2RGB)
image_front_rgb = cv2.cvtColor(image_front, cv2.COLOR_BGR2RGB)
image_top_next_day_rgb = cv2.cvtColor(image_top_next_day, cv2.COLOR_BGR2RGB)
image_front_next_day_rgb = cv2.cvtColor(image_front_next_day, cv2.COLOR_BGR2RGB)

# Применение пороговой обработки для выделения зеленых пикселей на изображении сверху
# (Здесь должны быть заданы lower_green и upper_green)
lower_green = np.array([0, 100, 0], dtype=np.uint8)
upper_green = np.array([50, 255, 50], dtype=np.uint8)

mask_top = cv2.inRange(image_top_rgb, lower_green, upper_green)
mask_front = cv2.inRange(image_front_rgb, lower_green, upper_green)
mask_top_next_day = cv2.inRange(image_top_next_day_rgb, lower_green, upper_green)
mask_front_next_day = cv2.inRange(image_front_next_day_rgb, lower_green, upper_green)

# Применение операции морфологического закрытия для удаления шума
kernel = np.ones((5, 5), np.uint8)
closed_top = cv2.morphologyEx(mask_top, cv2.MORPH_CLOSE, kernel)
closed_front = cv2.morphologyEx(mask_front, cv2.MORPH_CLOSE, kernel)
closed_top_next_day = cv2.morphologyEx(mask_top_next_day, cv2.MORPH_CLOSE, kernel)
closed_front_next_day = cv2.morphologyEx(mask_front_next_day, cv2.MORPH_CLOSE, kernel)

# Поиск контуров зеленых регионов
contours_top, _ = cv2.findContours(closed_top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_front, _ = cv2.findContours(closed_front, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_top_next_day, _ = cv2.findContours(closed_top_next_day, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_front_next_day, _ = cv2.findContours(closed_front_next_day, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание функции для сопоставления контуров
def match_contours(contours_curr, contours_next):
    matches = []
    for contour_curr in contours_curr:
        best_match = (None, None, float('inf'))
        for contour_next in contours_next:
            distance = cv2.matchShapes(contour_curr, contour_next, cv2.CONTOURS_MATCH_I1, 0.0)
            if distance < best_match[2]:
                best_match = (contour_curr, contour_next, distance)
        matches.append(best_match)
    return matches

# Создание тензоров для сохранения высот растков
heights_top = np.zeros(len(contours_top))
heights_front = np.zeros(len(contours_front))

# Измерение высот растков и сохранение в тензоры
for i, contour_top in enumerate(contours_top):
    height_top = cv2.contourArea(contour_top)
    heights_top[i] = height_top

for i, contour_front in enumerate(contours_front):
    height_front = cv2.contourArea(contour_front)
    heights_front[i] = height_front

# Вывод измеренных высот растков на первый день
print("\nВысоты растков на первый день (сверху):", heights_top)
print("Высоты растков на первый день (спереди):", heights_front)

# Сопоставление контуров между днями
contour_matches_top = match_contours(contours_top, contours_top_next_day)

# Создайте список для хранения результатов сопоставления
matches_results_top = []

# Задайте порог расстояния для считания контуров сопоставленными
matching_threshold = 0.1

# Вывод результатов сопоставления и формирование списка результатов
print("\nРезультаты сопоставления контуров (сверху):")
for match in contour_matches_top:
    contour_curr, contour_prev, distance = match
    is_matching = distance < matching_threshold
    matches_results_top.append(is_matching)
    print(f"Контур текущего дня: {contour_curr}, Контур предыдущего дня: {contour_prev}, Расстояние: {distance}, Совпадение: {is_matching}")

# Вывод таблицы результатов
print("\nТаблица результатов (сверху):")
for i, is_match in enumerate(matches_results_top):
    print(f"Растение {i + 1}: {is_match}")

# Укажите директорию, в которой вы хотите сохранить CSV файл
output_directory = "output_data"

# Создайте директорию, если ее нет
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Укажите путь к файлу внутри директории
csv_file_path = os.path.join(output_directory, "результаты.csv")

# Запись результатов в CSV файл
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Растение", "Совпадение"])
    for i, is_match in enumerate(matches_results_top):
        writer.writerow([f"Растение {i + 1}", is_match])

print(f"\nРезультаты сохранены в файл: {csv_file_path}")



Высоты растков на первый день (сверху): [  0.    0.    0.   53.5   3.    0.    0.    0.    0.    0.    0.    0.
   0.    4.5   0.    0.    0.  175.5   0.    0.    0.    0.    0.    0.
   7.5   0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  11.5   0.    1.5  28.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0. ]
Высоты растков на первый день (спереди): []

Результаты сопоставления контуров (сверху):
Контур текущего дня: [[[1035  484]]

 [[1035  485]]], Контур предыдущего дня: [[[1035  484]]

 [[1035  485]]], Расстояние: 0.0, Совпадение: True
Контур текущего дня: [[[201 468]]], Контур предыдущего дня: [[[1035  484]]

 [[1035  485]]], Расстояние: 0.0, Совпадение: True
Контур текущего дня: [[[641 466]]], Контур предыдущего дня: [[[1035  484]]

 [[1035  485]]], Расстояние: 0.0, Совпадение: True
Контур текущего дня: [[[625 443]]

 [[625 445]]

 [[624 446]]

 [[624 455]]

 [[623 456]]

 [[620